In [1]:
import os
import pandas as pd
import numpy as np
import kagglehub
import matplotlib.pyplot as plt
import seaborn as sns
import duckdb

sns.set(style="whitegrid")

In [2]:
path = kagglehub.dataset_download("vivekananda99/imdb-dataset")
print("Dataset downloaded to:", path)

Mounting files to /kaggle/input/imdb-dataset...
Dataset downloaded to: /kaggle/input/imdb-dataset


In [3]:
for root, dirs, files in os.walk(path):
    level = root.replace(path, "").count(os.sep)
    indent = " " * 4 * level
    print(f"{indent}{os.path.basename(root)}/")
    sub_indent = " " * 4 * (level + 1)
    for f in files:
        print(f"{sub_indent}{f}")

imdb-dataset/
    title.basics.tsv
    title.episode.tsv
    title.principals.tsv
    title.ratings.tsv
    name.basics.tsv
    title.akas.tsv
    title.crew.tsv


In [13]:
# %% [code]

file_path = f"{path}/title.basics.tsv"

con = duckdb.connect(database=":memory:")

con.execute(f"""
    CREATE OR REPLACE TABLE movies_filtered AS
    SELECT
        tconst,
        CAST(startYear AS INTEGER) AS startYear,
        genres
    FROM read_csv_auto('{file_path}', delim='\t', nullstr='\\N')
    WHERE titleType = 'movie'
      AND startYear IS NOT NULL
      AND CAST(startYear AS INTEGER) >= 1995
      AND genres IS NOT NULL
      AND genres != ''
""")

count = con.execute("SELECT COUNT(*) FROM movies_filtered").fetchone()[0]

print("Filtered movie count:", count)

con.execute("SELECT * FROM movies_filtered LIMIT 5").fetchdf()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Filtered movie count: 366304


,tconst,startYear,genres
0,tt0011801,2019,"Action,Crime"
1,tt0035423,2001,"Comedy,Fantasy,Romance"
2,tt0062336,2020,Drama
3,tt0067460,1995,"Documentary,Sport"
4,tt0069049,2018,Drama


In [14]:
# %% [code]

# Expand multi-genre rows
con.execute("""
    CREATE OR REPLACE TABLE movies_expanded AS
    SELECT
        tconst,
        startYear,
        TRIM(genre) AS genre
    FROM movies_filtered
    LEFT JOIN UNNEST(string_split(genres, ',')) AS g(genre)
    ON TRUE
""")

expanded_count = con.execute("SELECT COUNT(*) FROM movies_expanded").fetchone()[0]
print("Expanded rows:", expanded_count)

# Compute complexity (# genres per movie)
con.execute("""
    CREATE OR REPLACE TABLE movie_complexity AS
    SELECT
        tconst,
        COUNT(*) AS genre_count
    FROM movies_expanded
    GROUP BY tconst
""")

# Preview joined result
preview = con.execute("""
    SELECT
        e.tconst,
        e.genre,
        c.genre_count
    FROM movies_expanded e
    JOIN movie_complexity c USING (tconst)
    LIMIT 10
""").fetchdf()

preview

Expanded rows: 562530


,tconst,genre,genre_count
0,tt11143486,History,2
1,tt11138922,Documentary,2
2,tt11138922,History,2
3,tt11122932,Documentary,2
4,tt11122932,History,2
5,tt11120182,Documentary,2
6,tt11120182,History,2
7,tt1110287,Documentary,2
8,tt1110287,History,2
9,tt11048284,Documentary,2


In [15]:
# %% [code]

ratings_path = f"{path}/title.ratings.tsv"

con.execute(f"""
    CREATE OR REPLACE TABLE movies_enriched AS
    SELECT
        e.tconst,
        e.genre,
        e.startYear,
        c.genre_count,

        CAST(r.averageRating AS DOUBLE) AS averageRating,
        CAST(r.numVotes AS INTEGER) AS numVotes,

        -- normalize popularity
        CAST(r.numVotes AS DOUBLE)
            / ( {CURRENT_YEAR} - e.startYear + 1 ) AS votes_per_year,

        -- hybridity buckets
        CASE
            WHEN c.genre_count = 1 THEN 'Pure'
            WHEN c.genre_count = 2 THEN 'Hybrid-2'
            ELSE 'Hybrid-3+'
        END AS hybridity_bucket

    FROM movies_expanded e
    JOIN movie_complexity c USING (tconst)
    JOIN read_csv_auto('{ratings_path}', delim='\t', nullstr='\\N') r
        USING (tconst)

    WHERE r.averageRating IS NOT NULL
      AND r.numVotes IS NOT NULL
""")

final_count = con.execute("SELECT COUNT(*) FROM movies_enriched").fetchone()[0]

print("Final enriched rows:", final_count)

con.execute("""
    SELECT
        genre,
        hybridity_bucket,
        ROUND(AVG(averageRating), 2) AS avg_rating,
        ROUND(AVG(votes_per_year)) AS avg_votes_per_year
    FROM movies_enriched
    GROUP BY genre, hybridity_bucket
    LIMIT 10
""").fetchdf()

Final enriched rows: 365783


,genre,hybridity_bucket,avg_rating,avg_votes_per_year
0,Documentary,Hybrid-2,7.19,inf
1,Action,Hybrid-3+,5.74,inf
2,Family,Pure,6.09,inf
3,Mystery,Pure,6.28,inf
4,Music,Pure,7.53,inf
5,Documentary,Hybrid-3+,7.27,inf
6,Biography,Pure,6.85,inf
7,Romance,Hybrid-3+,6.08,inf
8,Adventure,Pure,6.18,inf
9,Animation,Hybrid-3+,6.33,inf


In [16]:
# %% [code]

# Build the correct movie-level enriched table

con.execute(f"""
    CREATE OR REPLACE TABLE movies_master AS
    SELECT
        b.tconst,
        b.startYear,
        b.genres,
        c.genre_count,

        CAST(r.averageRating AS DOUBLE) AS averageRating,
        CAST(r.numVotes AS INTEGER) AS numVotes,

        -- Safe normalized popularity
        CAST(r.numVotes AS DOUBLE) /
            CASE
                WHEN ({CURRENT_YEAR} - b.startYear + 1) <= 0 THEN 1
                ELSE ({CURRENT_YEAR} - b.startYear + 1)
            END AS votes_per_year,

        CASE
            WHEN c.genre_count = 1 THEN 'Pure'
            WHEN c.genre_count = 2 THEN 'Hybrid-2'
            ELSE 'Hybrid-3+'
        END AS hybridity_bucket

    FROM movies_filtered b

    JOIN movie_complexity c USING (tconst)

    JOIN read_csv_auto('{ratings_path}', delim='\t', nullstr='\\N') r
        USING (tconst)

    WHERE r.averageRating IS NOT NULL
      AND r.numVotes IS NOT NULL
""")

total_movies = con.execute("SELECT COUNT(*) FROM movies_master").fetchone()[0]

print("Movies in master table:", total_movies)

# validation: ensure no nulls or infinite values
check = con.execute("""
    SELECT
        SUM(CASE WHEN averageRating IS NULL THEN 1 ELSE 0 END) AS null_ratings,
        SUM(CASE WHEN numVotes IS NULL THEN 1 ELSE 0 END) AS null_votes,
        SUM(CASE WHEN votes_per_year IS NULL THEN 1 ELSE 0 END) AS null_votes_per_year,
        SUM(CASE WHEN votes_per_year = 'inf' THEN 1 ELSE 0 END) AS inf_votes_per_year
    FROM movies_master
""").fetchdf()

check

Movies in master table: 212595


,null_ratings,null_votes,null_votes_per_year,inf_votes_per_year
0,0.0,0.0,0.0,0.0


In [18]:
# %% [code]

def compute_genre_hybridity(genre: str):
    """
    Returns aggregated hybridity metrics for a given base genre.
    """

    query = f"""
        SELECT
            hybridity_bucket,

            COUNT(*) AS movie_count,

            ROUND(MEDIAN(averageRating), 3) AS median_rating,
            ROUND(MEDIAN(numVotes)) AS median_votes,
            ROUND(MEDIAN(votes_per_year), 2) AS median_votes_per_year,

            ROUND(STDDEV(averageRating), 3) AS rating_std

        FROM movies_master
        WHERE genres LIKE '%{genre}%'

        GROUP BY hybridity_bucket
        ORDER BY hybridity_bucket
    """

    return con.execute(query).fetchdf()


# Quick test
compute_genre_hybridity("Romance")

,hybridity_bucket,movie_count,median_rating,median_votes,median_votes_per_year,rating_std
0,Hybrid-2,9600,6.1,203.0,21.10,1.202
1,Hybrid-3+,8833,6.2,306.0,29.83,1.224
2,Pure,2349,6.2,44.0,5.86,1.389


In [19]:
# %% [code]

def summarize_genre(genre: str):

    df = compute_genre_hybridity(genre)

    best_rating_row = df.loc[df['median_rating'].idxmax()]
    best_engagement_row = df.loc[df['median_votes_per_year'].idxmax()]
    lowest_std_row = df.loc[df['rating_std'].idxmin()]

    summary = pd.DataFrame({
        "Metric": [
            "Best Complexity Tier",
            "Highest Median Rating",
            "Highest Engagement (Votes/Year)",
            "Lowest Polarization (Rating STD)"
        ],
        genre: [
            best_rating_row["hybridity_bucket"],
            best_rating_row["median_rating"],
            best_engagement_row["median_votes_per_year"],
            lowest_std_row["rating_std"]
        ]
    })

    return summary


def compare_genres(genre_a: str, genre_b: str):
    a = summarize_genre(genre_a)
    b = summarize_genre(genre_b)

    comparison = a.merge(b, on="Metric")

    return comparison


# Demo comparison
compare_genres("Horror", "Comedy")

,Metric,Horror,Comedy
0,Best Complexity Tier,Hybrid-3+,Hybrid-3+
1,Highest Median Rating,5.1,6.1
2,Highest Engagement (Votes/Year),47.48,36.0
3,Lowest Polarization (Rating STD),1.382,1.303


In [20]:
# %% [code]

export_path = "genre_hybridity_master.csv"

final_df = con.execute("""
    SELECT
        tconst,
        startYear,
        genres,
        genre_count,
        hybridity_bucket,
        ROUND(averageRating, 3) AS averageRating,
        numVotes,
        ROUND(votes_per_year, 2) AS votes_per_year
    FROM movies_master
""").fetchdf()

final_df.to_csv(export_path, index=False)

print("Export completed")
print("File name:", export_path)
print("Rows:", len(final_df))

final_df.head()

Export completed
File name: genre_hybridity_master.csv
Rows: 212595


,tconst,startYear,genres,genre_count,hybridity_bucket,averageRating,numVotes,votes_per_year
0,tt30700695,2024,"Drama,Romance",2,Hybrid-2,7.1,1237,1237.00
1,tt30700957,2024,"Horror,Thriller",2,Hybrid-2,2.6,250,250.00
2,tt30700968,2023,Documentary,1,Pure,7.0,69,34.50
3,tt3070106,2013,Thriller,1,Pure,6.5,89,7.42
4,tt30701187,2025,Crime,1,Pure,7.3,20,20.00
